In [7]:
# importing libraries
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

ModuleNotFoundError: No module named 'nltk'

In [21]:
# tokenizing and lematizing
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open(r'C:\Users\Xhensil\Downloads\intents(1).json').read()
intents = json.loads(data_file)

for intent in intents.get('intents', []):
    # Check if the intent has a 'patterns' key and it is a list
    if 'patterns' in intent and isinstance(intent['patterns'], list):
        # Iterate through each pattern in the list
        for pattern in intent['patterns']:
            # Tokenize each word
            w = nltk.word_tokenize(pattern)
            words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
# lemmatize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('texts.pkl','wb'))
pickle.dump(classes,open('labels.pkl','wb'))
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)

416 documents
389 classes ['What are the types of depression?', 'abandonment issues', 'about', 'academic stress', 'academic-pressure', 'academic-support', 'adult-intervention', 'afternoon', 'aging-acceptance', 'aging-fulfillment', 'aging-identity', 'aging-legacy', 'aging-reflections', 'aging-regrets', 'aging-relationships', 'aging-self-esteem', 'aging-wellness', 'anger management', 'anorexia nervosa', 'anxiety', 'anxious', 'ask', 'assertiveness', 'at what age does anxiety peak?', 'bad language', 'binge eating disorder', 'body image', 'body image struggles', 'body-image-acceptance', 'boundary-setting', 'budgeting', 'building a support network', 'building resilience', 'building-resilience', 'bulimia nervosa', 'bullying and discrimination', 'bullying-and-mental-health', 'bullying-awareness-week', 'bullying-prevention-month', 'bullying-recognition', 'bullying-reporting-system', 'can lack of sleep make you feel sad?', 'can low blood sugar cause suicidal thoughts?', 'career-transitions', 'ca

In [22]:

# training set, bag of word
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# 
# ]s for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

In [23]:

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


Training data created


C:\Users\xhensil\AppData\Local\Temp\ipykernel_49396\2001348992.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [24]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))


In [25]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


c:\Users\Xhensil\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [26]:
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('model.h5', hist)
print("model created")

Epoch 1/200
84/84 [==============================] - 2s 4ms/step - loss: 5.9751 - accuracy: 0.0000e+00
Epoch 2/200
84/84 [==============================] - 0s 4ms/step - loss: 5.9665 - accuracy: 0.0024
Epoch 3/200
84/84 [==============================] - 0s 3ms/step - loss: 5.9569 - accuracy: 0.0048
Epoch 4/200
84/84 [==============================] - 0s 4ms/step - loss: 5.9454 - accuracy: 0.0096
Epoch 5/200
84/84 [==============================] - 0s 4ms/step - loss: 5.9294 - accuracy: 0.0072
Epoch 6/200
84/84 [==============================] - 0s 4ms/step - loss: 5.9187 - accuracy: 0.0096
Epoch 7/200
84/84 [==============================] - 0s 5ms/step - loss: 5.9052 - accuracy: 0.0120
Epoch 8/200
84/84 [==============================] - 0s 6ms/step - loss: 5.8781 - accuracy: 0.0096
Epoch 9/200
84/84 [==============================] - 0s 5ms/step - loss: 5.8456 - accuracy: 0.0168
Epoch 10/200
84/84 [==============================] - 0s 4ms/step - loss: 5.7918 - accuracy: 0.0120
Epoch